In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import numpy as np
import pandas as pd
import os
import json
import random

from fastai.text import *
from fastai import *

In [3]:
path = Path('/home/jupyter/insight_project/Project-M/data/preprocessed/csv')
filenames = path.ls()
len(filenames)

72

In [3]:
#Export
class Interface():
    def __init__(self, path, bs=64, eval_mode=False):
        '''
        path must be a valid file path or file paths
        path can be either str or Path obj
        eval_mode set to False means create language model 
            and general classfier from scratch
        '''
        if not isinstance(path,(list,tuple)):
            path = self._listify(path)     #if things are not iteratorable
        self.path = self._clean_non_json_path(path) #remove non .json paths
        self.csv_path = None
        self.bs = bs
        self.language_model_data = None
        self.classification_model_data = None
        self.eval_mode = eval_mode
        self.df_train, self.df_valid = None, None
        self.data_list, self.dataset_name = [], [] #to store all individual df for individual model
        
    def _clean_non_json_path(self,path):
        filenames = []
        for file_path in path:
            file_extension = file_path.name.split('.')
            if len(file_extension) < 2: continue
            else:
                if file_path.name.split('.')[1] == 'json':
                    filenames.append(file_path)
        return filenames
    
    def _listify(self,p):
        '''Trun everying in p to a Path object'''
        
        if p is None: p = []
        elif isinstance(p,str): p = [p]
        else:
            if not isinstance(p,Iterable): p = [p]
        return [Path(i) for i in p]
    
    def _check_data_format(self):
        if self.path == []: return False
        one_file = self.path[0]
        flag = 0
        with open(one_file) as f:
            data = json.load(f)
        basilica_keys = ['data_points', 'num_data_points', 'id']
        baslilca_data_keys = ['body', 'label', 'source', 'embedding']
        for key in basilica_keys:
            if not key in data.keys(): flag +=1
        for key in baslilca_data_keys:
            if not key in data['data_points'][0].keys(): flag +=1 #only check the first item...
        if flag != 0:
            print('Warning: data source not from Basilica, exit...')
            return False
        return True
    
    def _convert_json_to_csv(self,test=False):
        #use self.path to grab json format Basilica data
        #return csv file
        #body,label,source
        body, label, source = [], [], []
        for one_file_path in self.path:
            with open(one_file_path) as file:
                data = json.load(file)
            for item in data['data_points']:
                body.append(item['body'])
                label.append(item['label'])
                source.append(item['source'])
            df = pd.DataFrame(list(zip(body, label, source)),
                             columns = ['Body', 'Label', 'Source'])
        
            #check if the parent directory exists
            if test:
                if not os.path.exists(one_file_path.parent/'delete_test'):
                    os.makedirs(one_file_path.parent/'delete_test')
            else:
                if not os.path.exists(one_file_path.parent/'csv'):
                    os.makedirs(one_file_path.parent/'csv')
        
            if test: dest = one_file_path.parent/'delete_test'
            else: dest = one_file_path.parent/'csv'
            filename = Path(one_file_path.name).stem + '.csv'
            df.to_csv(dest/filename,index=False)
            print(f'convert {filename} to csv')
        self.csv_path = dest
        
    def _prepare_df(self):
        dfs = []
        for file in self.csv_path.ls():
            df = pd.read_csv(file)
            dfs.append(df)
        return pd.concat(dfs)
    
    def _train_valid_split(self,total_filenames,train_pct=0.7):
        '''split the train and test base on total datasets
            return train_list and valid_list with their associated path'''
        total_filenames = total_filenames.ls()
        random.seed(42)
        random.shuffle(total_filenames)

        split_point = round(len(total_filenames)*train_pct)
        assert (type(split_point) == int)
        self.df_train, self.df_valid = total_filenames[:split_point],total_filenames[split_point:]
        
    def _get_lm(self,url):
        '''This code needs to be updated once I figure out how to host pretrained weights
            for now, it will use local path
        '''
        if url:
            data_lm = load_data(url,'data_lm_large.pkl',bs=self.bs)
        return data_lm
        
    def _get_individual_data(self,filepath,data_lm):
        df = pd.read_csv(filepath)
        if df.shape[0]*0.7 < self.bs: 
            bs = int(df.shape[0]*0.7)
        else: bs=self.bs
        data = (TextList
                .from_df(df=df,path=self.csv_path,cols='Body',vocab=data_lm.vocab)
                .split_by_rand_pct(0.3,seed=42)
                .label_from_df(cols='Label')
                .databunch(bs=bs,num_workers=os.cpu_count()*4)
               )
        return data
        
    
    def pre_processing(self,skip_convert_json=False,test=False,**kwargs):
        if not self._check_data_format(): exit(0) #we can't handle non Basilica data
        if not skip_convert_json: self._convert_json_to_csv(test)
        if self.eval_mode:
            if 'lm_path' in kwargs.keys():
                lm_path = kwargs['lm_path']
            else:
                print('Evaluation mode: lm_path not set...')
                exit(0)
            data_lm = self._get_lm(lm_path)
            for file in self.csv_path.ls():
                file_extension = Path(file).name.split('.')
                if len(file_extension) < 2 or file_extension[1] != 'csv': continue
                self.data_list.append(self._get_individual_data(file,data_lm))
                self.dataset_name.append(file_extension[0])
        else:
            '''
            Getting language model data bunch
            Split general classifier train and valid
            '''
            df_total = self._prepare_df()
            #Spacy Tokenization,pre_rules, post_rules applied,Numericalization
            #create fastai databunch, which is a pair of train, valid dataloader of Pytorch
            self.language_model_data = (TextList
                                        .from_df(df_total,path=self.csv_path)
                                        .split_by_rand_pct(0.1)
                                        .label_for_lm()
                                        .databunch(bs=self.bs,num_workers=16)
                                        )
            #Split data for general classifier
            self._train_valid_split(self.csv_path)
            
            
        

In [6]:
path = Path('/home/jupyter/insight_project/Project-M/data/preprocessed/')
path.ls()

[PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ed91c398-31c6-437f-a9d1-462e3ccfb6fa.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/example.txt'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ff063ea9-62b8-4f29-9faa-04a09cb5fba2.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/.ipynb_checkpoints'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/delete_test'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/fabab216-0767-4aa5-85fa-bb8852eb30d3.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/csv'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/f6d2081a-0f79-4abf-9021-c4d254859890.json')]

In [7]:
test = Interface(path.ls(),eval_mode=True)

In [8]:
test.path

[PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ed91c398-31c6-437f-a9d1-462e3ccfb6fa.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ff063ea9-62b8-4f29-9faa-04a09cb5fba2.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/fabab216-0767-4aa5-85fa-bb8852eb30d3.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/f6d2081a-0f79-4abf-9021-c4d254859890.json')]

In [9]:
test.pre_processing(test=True,lm_path='/home/jupyter/insight_project/Project-M/data/preprocessed/csv/models')

In [10]:
len(test.data_list)

4

In [11]:
test.dataset_name

['ed91c398-31c6-437f-a9d1-462e3ccfb6fa',
 'fabab216-0767-4aa5-85fa-bb8852eb30d3',
 'ff063ea9-62b8-4f29-9faa-04a09cb5fba2',
 'f6d2081a-0f79-4abf-9021-c4d254859890']

In [ ]:
#         if self.lm_retrain:
#             df_total = _prepare_df()
#             #Spacy Tokenization
#             #pre_rules, post_rules applied
#             #pre_rules: [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces]
#             #post_rules: [replace_all_caps, deal_caps]
#             #Numericalization
#             #create fastai databunch, which is a pair of train, valid dataloader of Pytorch
#             self.language_model_data = (TextList
#                                         .from_df(df_total,path=path)
#                                         .split_by_rand_pct(0.1)
#                                         .label_for_lm()
#                                         .databunch(bs=self.bs,num_workers=16)
#                                         )
#         else:
#             #Have a language model already
#             if self.language_model_path != None:
#                 data_lm_path = Path(self.language_model_path).parent
#             elif not os.path.exists(self.csv_path/'models'):
#                 print('Warning: No language model found... aborting...')
#                 exit(0)
#             else: data_lm_path = self.csv_path/'models'
#             self.language_model_data = load_data(data_lm_path, 'data_lm.pkl',bs=self.bs)
#         self.cls_model_data = (TextList
#                                .from_df()
#                               )

In [4]:
from notebook2script import *

In [5]:
notebook2script('Test_module.ipynb','interface')

Converted Test_module.ipynb to uti/interface.py
